In [3]:
import math

class RegConvexPolygon:
    '''
    Regular Convex Polygon class. A polygon is called "Regular Convex Polygon" if and only if:
        a. Equal Side Lengths: All the sides of the polygon are of equal length.
        b. Equal Angles: All the interior angles are equal.
        c. Convexity: The polygon is convex, meaning all its interior angles are less than 180 degrees, 
           and no part of the polygon "caves in."
    '''
    def __init__(self, edges: int, circum_radius: float, max_edges: int = 10) -> None:
        '''
        Initialize the convex polygon with its number of edges and circumradius and sentinel 
        value for end of iteration.
        '''
        # print("RegConvexPolygon __init__ is called")
        if not isinstance(circum_radius, (int, float)):
            raise TypeError('Polygon circumradius must be of type int or float')
        
        if circum_radius <= 0:
            raise ValueError('Circumradius must be a positive real number')

        if not isinstance(edges, int):
            raise TypeError('Edges must be of type int')
        if edges < 3:
            raise ValueError('Polygon must have at least 3 sides')
        if not isinstance(max_edges, int):
            raise TypeError('max_edges must be of type int')
        if max_edges < 3:
            raise ValueError('max_edges must not be less than 3')

        self.calculate_area = True
        self._circum_radius = circum_radius
        self._max_edges = max_edges
        self.edges = edges
        self._area = 0.0
    
    def __repr__(self):
        return f'Polygon(n={self._edges}, R={self._circum_radius})'

    # Returns number of vertices which is equal to edges
    @property
    def vertices(self):
        return self._edges
    
    # Calculates and returns the interior angle
    @property
    def interior_angle(self):
        return round((self._edges - 2) * (180 / self._edges), 2)
    
    # Calculates and returns the edge length
    @property
    def edge_len(self):
        return (round(2 * self._circum_radius * math.sin(math.radians(180 / self._edges)), 4))

    # Calculates and returns the apothem
    @property
    def apothem(self):
        return(round(self._circum_radius * math.cos(math.radians(180 / self._edges)), 4))
    
    # Calculates and returns the area. And area is computed only if number of edge is changed
    @property
    def area(self):
        if self.calculate_area is True:
            self._area = round(0.5 * self._edges * self.edge_len * self.apothem,4)
        return(self._area)
    
    # Calcuates and returns perimeter
    @property
    def perimeter(self):
        return(round(self._edges * self.edge_len, 4))
    
    # Returns the number of edges
    @property
    def edges(self):
        return self._edges
    
    # Sets the private variable for edges 
    @edges.setter
    def edges(self, edges):
        self.calculate_area = True
        self._edges = edges

    # Makes the object indexable
    def __getitem__(self, index):
        if not isinstance(index, int):
            raise TypeError ('Index has to have of type int')
        
        if index > self._max_edges:
            raise ValueError(f"RegConvexPolygon supports polygons up to {self._max_edges} edges")
        
        self.edges = index
        return self.area, self.perimeter
    
    def __iter__(self):
        # print("RegConvexPolygon __iter__ is called")
        return self.PolygonIterator(self)
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return (self.edges == other.edges 
                    and self._circum_radius == other._circum_radius)
        else:
            return NotImplemented
        
    def __gt__(self, other):
        if isinstance(other, self.__class__):
            return self.vertices > other.vertices
        else:
            return NotImplemented
    
    class PolygonIterator:
        ''' Iterator used by RegConvexPolygon to iterate over different number of edges
        '''
        def __init__(self, obj_reg_conv_poly):
            ''' Bind the iterable to local variable for future accessing
            '''
            # print("PolygonIterator __init__ is called")
            self.obj = obj_reg_conv_poly
            self.edge_idx = 3

        def __iter__(self):
            # print("PolygonIterator __iter__ is called")
            return self
        
        def __next__(self):
            ''' if edge is greater than sentinal value then stop iteration
            '''
            if self.edge_idx > self.obj._max_edges:
                raise StopIteration
            else:
                ''' Set the edges, which sets calculate_area to True
                '''
                # print("PolygonIterator __next__ is called")
                self.obj.edges = self.edge_idx
                ret_value = self.obj.area, self.obj.perimeter
                self.edge_idx =  self.edge_idx + 1
                return ret_value

In [5]:
def test_polygon():
    abs_tol = 0.001
    rel_tol = 0.001
    
    try:
        p = RegConvexPolygon(2, 10)
        assert False, ('Creating a Polygon with 2 sides: '
                       ' Exception expected, not received')
    except ValueError:
        pass

    n = 3
    R = 1
    p = RegConvexPolygon(n, R)
    assert str(p) == 'Polygon(n=3, R=1)', f'actual: {str(p)}'
    assert p.vertices == n, (f'actual: {p.vertices},'
                                   f' expected: {n}')
    assert p.edges == n, f'actual: {p.edges}, expected: {n}'
    assert p._circum_radius == R, f'actual: {p._circum_radius}, expected: {n}'
    assert p.interior_angle == 60, (f'actual: {p.interior_angle},'
                                    ' expected: 60')
    
    n = 4
    R = 1
    p = RegConvexPolygon(n, R)
    assert p.interior_angle == 90, (f'actual: {p.interior_angle}, '
                                    ' expected: 90')
    assert math.isclose(p.area, 2, 
                        rel_tol=abs_tol, 
                        abs_tol=abs_tol), (f'actual: {p.area},'
                                           ' expected: 2.0')
    
    assert math.isclose(p.edge_len, math.sqrt(2),
                       rel_tol=rel_tol,
                       abs_tol=abs_tol), (f'actual: {p.side_length},'
                                          f' expected: {math.sqrt(2)}')
    
    assert math.isclose(p.perimeter, 4 * math.sqrt(2),
                       rel_tol=rel_tol,
                       abs_tol=abs_tol), (f'actual: {p.perimeter},'
                                          f' expected: {4 * math.sqrt(2)}')
    
    assert math.isclose(p.apothem, 0.707,
                       rel_tol=rel_tol,
                       abs_tol=abs_tol), (f'actual: {p.perimeter},'
                                          ' expected: 0.707')
    
    p = RegConvexPolygon(6, 2)
    assert math.isclose(p.edge_len, 2,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.apothem, 1.73205,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.area, 10.3923,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.perimeter, 12,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.interior_angle, 120,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    
    p = RegConvexPolygon(12, 3)
    assert math.isclose(p.edge_len, 1.55291,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.apothem, 2.89778,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.area, 27,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.perimeter, 18.635,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    assert math.isclose(p.interior_angle, 150,
                        rel_tol=rel_tol, abs_tol=abs_tol)
    
    p1 = RegConvexPolygon(3, 10)
    p2 = RegConvexPolygon(10, 10)
    p3 = RegConvexPolygon(15, 10)
    p4 = RegConvexPolygon(15, 100)
    p5 = RegConvexPolygon(15, 100)
    
    assert p2 > p1
    assert p2 < p3
    assert p3 != p4
    assert p1 != p4
    assert p4 == p5

    print("ALL TESTS PASSED!")

test_polygon()

ALL TESTS PASSED!
